In [1]:
import sys
sys.path.insert(0, "/raid/mohamed/siim-isic-melanoma-classification")
from dataset.dataset_melanoma import DatasetMelanoma as Dataset
import torchvision.transforms as transforms
from augmentation.autoaugment import ImageNetPolicy
import pandas as pd

**Prepare dataset parameters**

In [2]:
image_dir = '../data/jpeg/train/'
train_csv = pd.read_csv('../data/train_split.csv')
mean = [104.00699, 116.66877, 122.67892]
std = [0.225*255, 0.224*255, 0.229*255]
transform = transforms.Compose([transforms.Resize((256, 256)), 
                                ImageNetPolicy(),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=mean, std=std)
                               ])

**Initialize dataset**

In [3]:
dataset = Dataset(df=train_csv,
                  image_dir=image_dir,
                  transform=transform)

**Test dataset**

In [4]:
(img, data), target = dataset[10000]

In [5]:
data

tensor([1.3542, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000])

In [6]:
data.shape

torch.Size([9])

**Test with the classifier**

In [7]:
from model.iternet.iternet_classifier import IternetClassifier
path = '../../cancer-segmentation/exp/iternet/_epoch_99.pth'
model = IternetClassifier(num_classes=2, path=path)

**Predict based on features and meta data**

In [8]:
x = img.unsqueeze(0)
y = model(x)

In [9]:
y.shape

torch.Size([1, 2])

In [10]:
y

tensor([[ 0.0996, -0.2161]], grad_fn=<AddmmBackward>)

In [11]:
import torch
torch.nn.Softmax(dim=1)(y)

tensor([[0.5783, 0.4217]], grad_fn=<SoftmaxBackward>)